In [1]:
import re
import os
from collections import Counter

from src.preprocesamiento.preprocesador import pre_procesar
from src.preprocesamiento.preprocesador import remover_puntuacion
from src.preprocesamiento.preprocesador import remover_acentos
from src.preprocesamiento.preprocesador import pasar_a_minusculas

os.environ["NLTK_DATA"]="datos_nltk/"
import  nltk
from nltk.corpus import stopwords

In [8]:
## Seleccionamos un corpus en español que se encuentra en nltk
from nltk.corpus import cess_esp

nltk.download("cess_esp")
corpus_tokens = cess_esp.words()

print(len(corpus_tokens))

[nltk_data] Downloading package cess_esp to datos_nltk/...
[nltk_data]   Package cess_esp is already up-to-date!


192686


In [9]:
# Dado nuestro corpus tokenizado, normalizamos el texto:
# Removiendo acentos, puntuacion y stopwords

nltk.download('stopwords')
stopwords_list = stopwords.words('spanish')

#print(stopwords_list)
tokens_normalizados = pre_procesar(corpus_tokens, stopwords_list)
len(tokens_normalizados)

[nltk_data] Downloading package stopwords to datos_nltk/...
[nltk_data]   Package stopwords is already up-to-date!


92164

In [13]:
## Obtenemos los tipos de nuestro corpus
tipos = Counter(tokens_normalizados)

print(len(tipos))

23900


In [15]:
print(tipos)

Counter({'0': 3889, 'fpa': 821, 'fpt': 820, 'hoy': 357, 'dos': 313, 'años': 291, 'si': 266, 'ser': 226, 'presidente': 223, 'gobierno': 222, 'segun': 204, 'millones': 195, 'dijo': 184, 'pais': 177, 'solo': 176, 'aunque': 175, 'puede': 163, 'año': 158, 'despues': 153, 'partido': 150, 'sido': 150, 'hace': 149, 'ahora': 146, 'equipo': 145, 'pasado': 141, 'tras': 136, 'tres': 134, 'mundo': 124, 'cada': 122, 'primera': 119, 'vida': 115, 'tiempo': 113, 'primer': 111, 'paises': 110, 'hacer': 108, 'mayor': 107, 'hacia': 107, 'parte': 103, 'españa': 103, 'dias': 103, 'gran': 103, 'ministro': 102, 'siempre': 100, 'tan': 100, 'politica': 96, 'dolares': 95, 'personas': 95, 'mismo': 94, 'vez': 89, 'bien': 86, 'mejor': 86, 'general': 86, 'cinco': 85, 'dia': 85, 'todas': 83, 'ademas': 83, 'meses': 81, 'haber': 81, 'grupo': 80, 'madrid': 80, 'sinembargo': 79, 'sino': 79, 'asi': 77, 'forma': 77, 'mercado': 77, 'caso': 76, 'hecho': 75, 'trabajo': 74, 'final': 74, 'cuatro': 74, 'ultimos': 74, 'yaque': 74,

In [5]:
# Dado nuestro corpus normalizado, procedemos a obtener los tipos
minusculas = pasar_a_minusculas(corpus_tokens)
#sin_acentos = remover_acentos(minusculas)
#sin_puntuacion = remover_puntuacion(sin_acentos)

len(minusculas)

#tipos = Counter(sin_puntuacion)

#tipos.most_common(100)


192686

In [6]:
sin_acentos = remover_acentos(minusculas)
len(sin_acentos)

192686

In [7]:
sin_puntuacion = remover_puntuacion(sin_acentos)
len(sin_puntuacion)

170224

In [5]:
tipos.most_common(100)


[('', 22462)]

In [2]:
esto = "esto es áéíóú \""
regex = r'[^a-z0-9ñáéíóúü]+'
print(re.sub(regex, '', esto))
print(remover_puntuacion(esto.split()))

estoesáéíóú
['esto', 'es', 'áéíóú']
